#### single levev json flattening

In [0]:
#read json from text file
jsonString="""{"Zipcode":3000,"ZipCodeType":"AusPost","City":"Melbourne","State":"VIC"}"""
df=spark.createDataFrame([(1, jsonString)],["id","value"])
df.show(truncate=False)

#Create schema for JSON
from pyspark.sql.types import StructType,StructField, StringType
schema = StructType([ 
    StructField("Zipcode",StringType(),True), 
    StructField("ZipCodeType",StringType(),True), 
    StructField("City",StringType(),True), 
    StructField("State", StringType(), True)
  ])

#Convert JSON string column to struct type
from pyspark.sql.functions import col,from_json
df2 = df.withColumn("json_flatten_data",from_json(col("value"),schema))

#Convert to multile columns
df3=df2.select("id", "json_flatten_data.*")
# df4.printSchema()
df3.show()


+---+-------------------------------------------------------------------------+
|id |value                                                                    |
+---+-------------------------------------------------------------------------+
|1  |{"Zipcode":3000,"ZipCodeType":"AusPost","City":"Melbourne","State":"VIC"}|
+---+-------------------------------------------------------------------------+

+---+-------+-----------+---------+-----+
| id|Zipcode|ZipCodeType|     City|State|
+---+-------+-----------+---------+-----+
|  1|   3000|    AusPost|Melbourne|  VIC|
+---+-------+-----------+---------+-----+



#### multi-level json with explode for array type and * for strut type element

In [0]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import explode, col

In [0]:
json_str ="""
{
        "id": "0001",
        "type": "donut",
        "name": "Cake",
        "ppu": 0.55,
        "batters":
                {
                        "batter":
                                [
                                        { "id": "1001", "type": "Regular" },
                                        { "id": "1002", "type": "Chocolate" },
                                        { "id": "1003", "type": "Blueberry" },
                                        { "id": "1004", "type": "Devil's Food" }
                                ]
                },
        "topping":
                [
                        { "id": "5001", "type": "None" },
                        { "id": "5002", "type": "Glazed" },
                        { "id": "5005", "type": "Sugar" },
                        { "id": "5007", "type": "Powdered Sugar" },
                        { "id": "5006", "type": "Chocolate with Sprinkles" },
                        { "id": "5003", "type": "Chocolate" },
                        { "id": "5004", "type": "Maple" }
                ]
}
"""

In [0]:
json_str ="""
[
{
        "id": "0001",
        "type": "donut",
        "name": "Cake",
        "ppu": 0.55,
        "batters":
                {
                        "batter":
                                [
                                        { "id": "1001", "type": "Regular" },
                                        { "id": "1002", "type": "Chocolate" },
                                        { "id": "1003", "type": "Blueberry" },
                                        { "id": "1004", "type": "Devil's Food" }
                                ]
                },
        "topping":
                [
                        { "id": "5001", "type": "None" },
                        { "id": "5002", "type": "Glazed" },
                        { "id": "5005", "type": "Sugar" },
                        { "id": "5007", "type": "Powdered Sugar" },
                        { "id": "5006", "type": "Chocolate with Sprinkles" },
                        { "id": "5003", "type": "Chocolate" },
                        { "id": "5004", "type": "Maple" }
                ]
}
,
{
        "id": "0002",
        "type": "cake",
        "name": "Cup_cake",
        "ppu": 0.55,
        "batters":
                {
                        "batter":
                                [
                                        { "id": "1001", "type": "Regular" },
                                        { "id": "1002", "type": "Chocolate" },
                                        { "id": "1003", "type": "Blueberry" },
                                        { "id": "1004", "type": "Devil's Food" }
                                ]
                },
        "topping":
                [
                        { "id": "5001", "type": "None" },
                        { "id": "5002", "type": "Glazed" },
                        { "id": "5005", "type": "Sugar" },
                        { "id": "5007", "type": "Powdered Sugar" },
                        { "id": "5006", "type": "Chocolate with Sprinkles" },
                        { "id": "5003", "type": "Chocolate" },
                        { "id": "5004", "type": "Maple" }
                ]
}

]
"""

In [0]:
df = spark.read.json(sc.parallelize([json_str]))
df.printSchema()

root
 |-- batters: struct (nullable = true)
 |    |-- batter: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ppu: double (nullable = true)
 |-- topping: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- type: string (nullable = true)



In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import explode, col

#Flatten array of structs and structs
def json_data_flatten(df):
  # compute Complex Fields (Lists and Structs) in Schema
  complex_fields = dict([(field.name, field.dataType)
                           for field in df.schema.fields
                           if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
  while len(complex_fields)!=0:
    col_name=list(complex_fields.keys())[0]
#     print ("Processing :"+col_name+" Type : "+str(type(complex_fields[col_name])))

    # if StructType then convert all sub element to columns.
    # i.e. flatten structs
    if (type(complex_fields[col_name]) == StructType):
      expanded = [col(col_name+'.'+k).alias(col_name+'_'+k) for k in [ n.name for n in  complex_fields[col_name]]]
      df=df.select("*", *expanded).drop(col_name)

    # if ArrayType then add the Array Elements as Rows using the explode function
    # i.e. explode Arrays
    elif (type(complex_fields[col_name]) == ArrayType):
      df=df.withColumn(col_name,explode_outer(col_name))

    # recompute remaining Complex Fields in Schema       
    complex_fields = dict([(field.name, field.dataType)
                           for field in df.schema.fields
                           if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
  return df


In [0]:
df_flatten =json_data_flatten(df)
display(df_flatten)

id,name,ppu,type,topping_id,topping_type,batters_batter_id,batters_batter_type
0001,Cake,0.55,donut,5001,None,1001,Regular
0001,Cake,0.55,donut,5001,None,1002,Chocolate
0001,Cake,0.55,donut,5001,None,1003,Blueberry
0001,Cake,0.55,donut,5001,None,1004,Devil's Food
0001,Cake,0.55,donut,5002,Glazed,1001,Regular
0001,Cake,0.55,donut,5002,Glazed,1002,Chocolate
0001,Cake,0.55,donut,5002,Glazed,1003,Blueberry
0001,Cake,0.55,donut,5002,Glazed,1004,Devil's Food
0001,Cake,0.55,donut,5005,Sugar,1001,Regular
0001,Cake,0.55,donut,5005,Sugar,1002,Chocolate


#### multi-level json with explode for array type and * for strut type element to get specific part of json file
#### explode will add new row for each element and column name will be col by default

In [0]:
df.printSchema()

root
 |-- batters: struct (nullable = true)
 |    |-- batter: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ppu: double (nullable = true)
 |-- topping: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- type: string (nullable = true)



In [0]:
# get only all "batters" - get all struct and explode the array and get all elements from exploded by using col.*
display(df.select("batters.*").select(explode("batter")).select("col.*")
       .dropDuplicates(['id', 'type'])
       )

id,type
1003,Blueberry
1004,Devil's Food
1001,Regular
1002,Chocolate


In [0]:
# get only all toppings
display(df.select(explode("topping")).select("col.*").sort('id'))

id,type
5001,None
5001,None
5002,Glazed
5002,Glazed
5003,Chocolate
5003,Chocolate
5004,Maple
5004,Maple
5005,Sugar
5005,Sugar


In [0]:
display(df.select("id" ,"name","batters","ppu",explode("topping")) #// Exploding the topping column using explode as it is an array type
        .withColumn("topping_id",col("col.id")) #// Extracting topping_id from col using DOT form
        .withColumn("topping_type",col("col.type")) #// Extracting topping_tytpe from col using DOT form
        .drop("col")
        .select("*","batters.*") #// Flattened the struct type batters to array type which is batter
        .drop("batters")
        .select("*",explode("batter"))
        .drop("batter")
        .withColumn("batter_id",col("col.id")) #// Extracting batter_id from col using DOT form
        .withColumn("battter_type",col("col.type")) #// Extracting battter_type from col using DOT form
        .drop("col")
       )

id,name,ppu,topping_id,topping_type,batter_id,battter_type
0001,Cake,0.55,5001,None,1001,Regular
0001,Cake,0.55,5001,None,1002,Chocolate
0001,Cake,0.55,5001,None,1003,Blueberry
0001,Cake,0.55,5001,None,1004,Devil's Food
0001,Cake,0.55,5002,Glazed,1001,Regular
0001,Cake,0.55,5002,Glazed,1002,Chocolate
0001,Cake,0.55,5002,Glazed,1003,Blueberry
0001,Cake,0.55,5002,Glazed,1004,Devil's Food
0001,Cake,0.55,5005,Sugar,1001,Regular
0001,Cake,0.55,5005,Sugar,1002,Chocolate


### get part of the json as separate file

In [0]:
new_json_data = """
{
    "results": [
        {
            "user": {
                "gender": "female",
                "name": {
                    "title": "miss",
                    "first": "lola",
                    "last": "masson"
                },
                "location": {
                    "street": "4670 place de la mairie",
                    "city": "montreuil",
                    "state": "charente-maritime",
                    "zip": 48774
                },
                "email": "lola.masson@example.com",
                "username": "organicbird989",
                "password": "gary",
                "salt": "uMFla5oI",
                "md5": "6e6df2bb8e7241f9e49d51fb5c2bc5bb",
                "sha1": "5e5bb0fa91e86db05c071599e7b8c4021f96f7e5",
                "sha256": "d06f91eaa7042ca35b9a19d8163396ca5401a2e02ab18aa62e7e4b746695f241",
                "registered": 1164508789,
                "dob": 738038639,
                "phone": "05-48-61-06-40",
                "cell": "06-47-24-19-61",
                "INSEE": "2930504297111 17",
                "picture": {
                    "large": "https://randomuser.me/api/portraits/women/93.jpg",
                    "medium": "https://randomuser.me/api/portraits/med/women/93.jpg",
                    "thumbnail": "https://randomuser.me/api/portraits/thumb/women/93.jpg"
                }
            }
        },
        {
            "user": {
                "gender": "male",
                "name": {
                    "title": "mr",
                    "first": "fabien",
                    "last": "nguyen"
                },
                "location": {
                    "street": "8812 rue du bât-d'argent",
                    "city": "rennes",
                    "state": "eure-et-loir",
                    "zip": 20771
                },
                "email": "fabien.nguyen@example.com",
                "username": "heavywolf152",
                "password": "wilder",
                "salt": "yVMf9oxL",
                "md5": "ea1cde87d1278a81248518d8941e1e14",
                "sha1": "2806dae56094157bdf4802f415f0c5c4cd89e463",
                "sha256": "3479179f8fb9a2bc2617a88eb7aded501528c5db9417003069c786653c6dbf9b",
                "registered": 1370449407,
                "dob": 148511403,
                "phone": "05-58-29-86-96",
                "cell": "06-35-75-56-51",
                "INSEE": "1740930947099 03",
                "picture": {
                    "large": "https://randomuser.me/api/portraits/men/64.jpg",
                    "medium": "https://randomuser.me/api/portraits/med/men/64.jpg",
                    "thumbnail": "https://randomuser.me/api/portraits/thumb/men/64.jpg"
                }
            }
        },
        {
            "user": {
                "gender": "female",
                "name": {
                    "title": "miss",
                    "first": "sophie",
                    "last": "carpentier"
                },
                "location": {
                    "street": "4085 place de l'europe",
                    "city": "boulogne-billancourt",
                    "state": "pyrénées-orientales",
                    "zip": 31486
                },
                "email": "sophie.carpentier@example.com",
                "username": "tinypanda983",
                "password": "peekaboo",
                "salt": "6tJqCcDn",
                "md5": "f1ee887fbab209f5f9748b0aa19395b1",
                "sha1": "a354e57b341b797bd7792d953b33a3b8ac33aad7",
                "sha256": "759f659a0607881edb87be1246b15e6f165195dbc5bf4b9601e6498bb22e0775",
                "registered": 1400827816,
                "dob": 905356670,
                "phone": "05-64-03-67-01",
                "cell": "06-00-95-79-97",
                "INSEE": "2980946821292 65",
                "picture": {
                    "large": "https://randomuser.me/api/portraits/women/16.jpg",
                    "medium": "https://randomuser.me/api/portraits/med/women/16.jpg",
                    "thumbnail": "https://randomuser.me/api/portraits/thumb/women/16.jpg"
                }
            }
        },
        {
            "user": {
                "gender": "female",
                "name": {
                    "title": "miss",
                    "first": "marilou",
                    "last": "moulin"
                },
                "location": {
                    "street": "5334 rue gasparin",
                    "city": "versailles",
                    "state": "alpes-maritimes",
                    "zip": 81080
                },
                "email": "marilou.moulin@example.com",
                "username": "greensnake819",
                "password": "scooter1",
                "salt": "9iiu2X7w",
                "md5": "9262f60a8312d905e75d0bb73435f661",
                "sha1": "85310aa2c2458f7447cbd3e25d1bc8f272f90c41",
                "sha256": "c42b6db56b1701c8f92420e7276deef50dc80c13c196f560f86f5bd95f4e33f8",
                "registered": 950855863,
                "dob": 157066596,
                "phone": "01-74-62-68-79",
                "cell": "06-77-58-92-25",
                "INSEE": "2741259261505 55",
                "picture": {
                    "large": "https://randomuser.me/api/portraits/women/91.jpg",
                    "medium": "https://randomuser.me/api/portraits/med/women/91.jpg",
                    "thumbnail": "https://randomuser.me/api/portraits/thumb/women/91.jpg"
                }
            }
        },
        {
            "user": {
                "gender": "male",
                "name": {
                    "title": "mr",
                    "first": "nathanaël",
                    "last": "durand"
                },
                "location": {
                    "street": "7233 rue du cardinal-gerlier",
                    "city": "tourcoing",
                    "state": "guyane",
                    "zip": 24179
                },
                "email": "nathanaël.durand@example.com",
                "username": "blackkoala288",
                "password": "door",
                "salt": "R6OHAh8d",
                "md5": "4b58ed523a6a1cc758f7839f599f9fed",
                "sha1": "65827d27a6679b45158dae9e0854c02cf913691e",
                "sha256": "d761d5af91de90c4b0fc9bf83fcdaf0157e7a84428242f79d47fa1dcb44139f1",
                "registered": 1231288642,
                "dob": 964173035,
                "phone": "05-75-34-13-35",
                "cell": "06-78-28-25-08",
                "INSEE": "1000728360862 01",
                "picture": {
                    "large": "https://randomuser.me/api/portraits/men/73.jpg",
                    "medium": "https://randomuser.me/api/portraits/med/men/73.jpg",
                    "thumbnail": "https://randomuser.me/api/portraits/thumb/men/73.jpg"
                }
            }
        },
        {
            "user": {
                "gender": "female",
                "name": {
                    "title": "miss",
                    "first": "lila",
                    "last": "arnaud"
                },
                "location": {
                    "street": "7347 rue de l'abbé-migne",
                    "city": "le mans",
                    "state": "haute-corse",
                    "zip": 37573
                },
                "email": "lila.arnaud@example.com",
                "username": "bigfish274",
                "password": "haha",
                "salt": "OENvStiY",
                "md5": "466b7daf829a37e705db55ff26eca98e",
                "sha1": "086767fad544ff5594e640504cfc6f361cd6a264",
                "sha256": "dac0ac166004350a8891e2368eac36fbe636db4c6d2deae3d37b530f33f19a20",
                "registered": 1343061224,
                "dob": 284780273,
                "phone": "03-40-34-71-86",
                "cell": "06-98-27-49-90",
                "INSEE": "2790167033329 81",
                "picture": {
                    "large": "https://randomuser.me/api/portraits/women/51.jpg",
                    "medium": "https://randomuser.me/api/portraits/med/women/51.jpg",
                    "thumbnail": "https://randomuser.me/api/portraits/thumb/women/51.jpg"
                }
            }
        },
        {
            "user": {
                "gender": "male",
                "name": {
                    "title": "mr",
                    "first": "nino",
                    "last": "nguyen"
                },
                "location": {
                    "street": "5635 rue du cardinal-gerlier",
                    "city": "marseille",
                    "state": "jura",
                    "zip": 35561
                },
                "email": "nino.nguyen@example.com",
                "username": "yellowtiger640",
                "password": "pacino",
                "salt": "twEHDKxd",
                "md5": "431ea524deac858bae3f2a98bac831e4",
                "sha1": "fb28cfcab27942c406e2983019e403e55de36502",
                "sha256": "de0288b57d0349aa128aa8c2e44e37504d76d99604e6a251dc1eae5ee84848b3",
                "registered": 1361592295,
                "dob": 932717995,
                "phone": "02-28-13-15-42",
                "cell": "06-42-33-31-57",
                "INSEE": "1990700107400 16",
                "picture": {
                    "large": "https://randomuser.me/api/portraits/men/54.jpg",
                    "medium": "https://randomuser.me/api/portraits/med/men/54.jpg",
                    "thumbnail": "https://randomuser.me/api/portraits/thumb/men/54.jpg"
                }
            }
        },
        {
            "user": {
                "gender": "female",
                "name": {
                    "title": "mrs",
                    "first": "sélène",
                    "last": "leclercq"
                },
                "location": {
                    "street": "7266 avenue joliot curie",
                    "city": "tours",
                    "state": "loire",
                    "zip": 65176
                },
                "email": "sélène.leclercq@example.com",
                "username": "goldenwolf216",
                "password": "pirates",
                "salt": "hhbn7de4",
                "md5": "97d45f1e2b63f016a9369caf9a3721d4",
                "sha1": "de7488342a1afb9b5a2fdbd916f8b9cfd36ddfdf",
                "sha256": "c8928117679a38962ae0330e1d8db4d4142a219729030b06ffc35fd7ec68fb03",
                "registered": 1166827498,
                "dob": 344293065,
                "phone": "04-54-87-39-06",
                "cell": "06-63-04-71-61",
                "INSEE": "2801111525740 44",
                "picture": {
                    "large": "https://randomuser.me/api/portraits/women/23.jpg",
                    "medium": "https://randomuser.me/api/portraits/med/women/23.jpg",
                    "thumbnail": "https://randomuser.me/api/portraits/thumb/women/23.jpg"
                }
            }
        },
        {
            "user": {
                "gender": "male",
                "name": {
                    "title": "mr",
                    "first": "timothe",
                    "last": "masson"
                },
                "location": {
                    "street": "4842 rue de la baleine",
                    "city": "versailles",
                    "state": "ardennes",
                    "zip": 56870
                },
                "email": "timothe.masson@example.com",
                "username": "orangelion503",
                "password": "geneva",
                "salt": "DERFFnEu",
                "md5": "eefc1e70c14ac6599bff90b9027799c8",
                "sha1": "243a6264203e9b2217b02268bc2c25086bd61bb3",
                "sha256": "20b5e2ef21c7b334f9e05a34113899a557eaf6ac03013561109bba038e877cf0",
                "registered": 1113134760,
                "dob": 272377358,
                "phone": "02-96-29-99-17",
                "cell": "06-33-56-56-58",
                "INSEE": "1780859418798 91",
                "picture": {
                    "large": "https://randomuser.me/api/portraits/men/75.jpg",
                    "medium": "https://randomuser.me/api/portraits/med/men/75.jpg",
                    "thumbnail": "https://randomuser.me/api/portraits/thumb/men/75.jpg"
                }
            }
        },
        {
            "user": {
                "gender": "male",
                "name": {
                    "title": "mr",
                    "first": "mathis",
                    "last": "laurent"
                },
                "location": {
                    "street": "5099 rue saint-georges",
                    "city": "courbevoie",
                    "state": "aude",
                    "zip": 79293
                },
                "email": "mathis.laurent@example.com",
                "username": "redfish768",
                "password": "becker",
                "salt": "yFfMS43W",
                "md5": "eeb41035ca0fd076f40f125ae2152f56",
                "sha1": "c7f8ef80fef9579bf03b9b23b1e61aeb40b59ab9",
                "sha256": "b1d935f6801f0c4f7353f276fd8da8332bb3e220a686463c61187ef96b78a18e",
                "registered": 1381457241,
                "dob": 1404245935,
                "phone": "01-49-40-97-92",
                "cell": "06-08-02-24-45",
                "INSEE": "1140730673664 56",
                "picture": {
                    "large": "https://randomuser.me/api/portraits/men/74.jpg",
                    "medium": "https://randomuser.me/api/portraits/med/men/74.jpg",
                    "thumbnail": "https://randomuser.me/api/portraits/thumb/men/74.jpg"
                }
            }
        }
    ],
    "nationality": "FR",
    "seed": "2ab3a0579f3488f007",
    "version": "0.8"
}
"""

In [0]:

df = spark.read.json(sc.parallelize([new_json_data]))
df.printSchema()

root
 |-- nationality: string (nullable = true)
 |-- results: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- user: struct (nullable = true)
 |    |    |    |-- INSEE: string (nullable = true)
 |    |    |    |-- cell: string (nullable = true)
 |    |    |    |-- dob: long (nullable = true)
 |    |    |    |-- email: string (nullable = true)
 |    |    |    |-- gender: string (nullable = true)
 |    |    |    |-- location: struct (nullable = true)
 |    |    |    |    |-- city: string (nullable = true)
 |    |    |    |    |-- state: string (nullable = true)
 |    |    |    |    |-- street: string (nullable = true)
 |    |    |    |    |-- zip: long (nullable = true)
 |    |    |    |-- md5: string (nullable = true)
 |    |    |    |-- name: struct (nullable = true)
 |    |    |    |    |-- first: string (nullable = true)
 |    |    |    |    |-- last: string (nullable = true)
 |    |    |    |    |-- title: string (nullable = true)
 |    |    |  

In [0]:
# get only all "user address locations" - get all struct and explode the array and get all elements from exploded by using col.*
display(df.withColumn("results",explode("results")).select(col("results.user.location.*"))
       .dropDuplicates(['city', 'state','street','zip'])
       )

city,state,street,zip
le mans,haute-corse,7347 rue de l'abbé-migne,37573
versailles,alpes-maritimes,5334 rue gasparin,81080
courbevoie,aude,5099 rue saint-georges,79293
montreuil,charente-maritime,4670 place de la mairie,48774
rennes,eure-et-loir,8812 rue du bât-d'argent,20771
boulogne-billancourt,pyrénées-orientales,4085 place de l'europe,31486
tourcoing,guyane,7233 rue du cardinal-gerlier,24179
marseille,jura,5635 rue du cardinal-gerlier,35561
tours,loire,7266 avenue joliot curie,65176
versailles,ardennes,4842 rue de la baleine,56870


#### second version of code

In [0]:
from pyspark.sql import types as T
import pyspark.sql.functions as F


def json_flatten(df):
    complex_fields = dict([
        (field.name, field.dataType) 
        for field in df.schema.fields 
        if isinstance(field.dataType, T.ArrayType) or isinstance(field.dataType, T.StructType)
    ])
    
    qualify = list(complex_fields.keys())[0] + "_"

    while len(complex_fields) != 0:
        col_name = list(complex_fields.keys())[0]
        
        if isinstance(complex_fields[col_name], T.StructType):
            expanded = [F.col(col_name + '.' + k).alias(col_name + '_' + k) 
                        for k in [ n.name for n in  complex_fields[col_name]]
                       ]
            
            df = df.select("*", *expanded).drop(col_name)
    
        elif isinstance(complex_fields[col_name], T.ArrayType): 
            df = df.withColumn(col_name, F.explode(col_name))
    
      
        complex_fields = dict([
            (field.name, field.dataType)
            for field in df.schema.fields
            if isinstance(field.dataType, T.ArrayType) or isinstance(field.dataType, T.StructType)
        ])
        
        
    for df_col_name in df.columns:
        df = df.withColumnRenamed(df_col_name, df_col_name.replace(qualify, ""))

    return df

In [0]:
df = json_flatten(df)
display(df)

id,name,ppu,type,topping_id,topping_type,batter_id,batter_type
0001,Cake,0.55,donut,5001,None,1001,Regular
0001,Cake,0.55,donut,5001,None,1002,Chocolate
0001,Cake,0.55,donut,5001,None,1003,Blueberry
0001,Cake,0.55,donut,5001,None,1004,Devil's Food
0001,Cake,0.55,donut,5002,Glazed,1001,Regular
0001,Cake,0.55,donut,5002,Glazed,1002,Chocolate
0001,Cake,0.55,donut,5002,Glazed,1003,Blueberry
0001,Cake,0.55,donut,5002,Glazed,1004,Devil's Food
0001,Cake,0.55,donut,5005,Sugar,1001,Regular
0001,Cake,0.55,donut,5005,Sugar,1002,Chocolate
